In [248]:
import calendar
import os
import pandas as pd
from python.save_csv import save_df_to_csv
from datetime import timedelta, datetime

In [249]:
# temp read from temp files

DATA_DIR = r'K:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/PYTHON/Scripts/OEE2/python/data'

def read_csv(filename):
    path = os.path.join(DATA_DIR, filename)
    return pd.read_csv(path)

df_circ_data = read_csv('df_circ_data.csv')
df_meters = read_csv('df_meters.csv')
df_operations = read_csv('df_operations.csv')
df_osi_operations = read_csv('df_osi_operations.csv')
df_rigs = read_csv('df_rigs.csv')

In [250]:
df_osi_operations

,equipment,event,starttime,endtime
0,DBKAZ40 BL №1,RP_REPAIR,2036-08-19 11:00:00,2036-09-01 16:00:00
1,DBKAZ40 BL №1,CV_LONG_STOP,2035-04-27 07:00:00,2035-08-15 07:00:00
2,DBKAZ40 BL №1,WT_PPR,2035-02-17 07:00:00,2035-02-17 08:00:00
3,DBKAZ40 BL №1,WT_LUNCH,2035-02-17 11:00:00,2035-02-17 12:00:00
4,DBKAZ40 BL №1,WT_PPR,2035-02-17 19:00:00,2035-02-17 20:00:00
...,...,...,...,...
567646,ЯЯЯЯ-ФЕЙК,DR_CORE_DRILLING,2039-09-29 20:00:00,2039-09-29 22:30:00
567647,ЯЯЯЯ-ФЕЙК,DR_REAMING,2039-09-30 10:30:00,2039-09-30 12:30:00
567648,ЯЯЯЯ-ФЕЙК,DR_PUMP_REPLACEMENT,2039-09-30 13:00:00,2039-09-30 14:30:00
567649,ЯЯЯЯ-ФЕЙК,DR_WELL_FLUSHING,2039-10-01 11:30:00,2039-10-01 14:00:00


In [251]:
df_meters

,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH
0,MSK39_08_09_A4,BurGeoProekt,PRAKLA_01-BGP,PRODUCT,ACCEPTED,2020-04-05,508.0
1,MSK03_06_02_A_,BurGeoProekt,PRAKLA_03-BGP,PRODUCT,LIQUID,2020-04-13,475.0
2,MSK03_04_05_A_,BurGeoProekt,PRAKLA_04-BGP,PRODUCT,LIQUID,2020-09-16,503.0
3,MSK03_04_09_A_,BurGeoProekt,PRAKLA_04-BGP,PRODUCT,LIQUID,2020-09-17,506.0
4,MSK03_08_07_A_,BurGeoProekt,PRAKLA_03-BGP,PRODUCT,ACCEPTED,2020-09-26,507.0
...,...,...,...,...,...,...,...
7202,TNU37_07_07_B_,TechnoService-Eng,ZIF1200_04-TSE,INJECT,ACCEPTED,2025-06-16,293.0
7203,TNU37_01_12_A_,TechnoService-Eng,ZIF1200_03-TSE,INJECT,ACCEPTED,2025-06-23,312.0
7204,TNU37_10_14_A_,TechnoService-Eng,ZIF1200_04-TSE,PRODUCT,ACCEPTED,2025-06-25,315.0
7205,TNU37_11_08_A_,TechnoService-Eng,ZIF1200_14-TSE,INJECT,ACCEPTED,2025-06-28,305.0


In [252]:
# Преобразуем Starttime и Endtime в datetime
if df_osi_operations is not None:
    df_osi_operations['starttime'] = pd.to_datetime(df_osi_operations['starttime'], errors='coerce')
    df_osi_operations['endtime'] = pd.to_datetime(df_osi_operations['endtime'], errors='coerce')

In [253]:
df_meters['ENDDATE'] = pd.to_datetime(df_meters['ENDDATE'])
mask1 = df_meters['ENDDATE'] > '2025.03.31'
df_meters = df_meters[mask1]


In [254]:
# Функция для преобразования имён колонок:
def clean_column_names(columns):
    return [col.replace('_', ' ').title() for col in columns]

# Очищаем и переименовываем колонки сразу после чтения
if df_circ_data is not None:
    df_circ_data.columns = clean_column_names(df_circ_data.columns)

if df_meters is not None:
    df_meters.columns = clean_column_names(df_meters.columns)
    
if df_operations is not None:   
    df_operations.columns = clean_column_names(df_operations.columns)
    
if df_osi_operations is not None:  
    df_osi_operations.columns = clean_column_names(df_osi_operations.columns)
    
if df_rigs is not None:
    df_rigs.columns = clean_column_names(df_rigs.columns)

In [255]:
def add_difference_column(df, col1, col2):
    """
    Возвращает Series: разница между двумя колонками с заменой NaN на 0.
    """
    return (df[col1].fillna(0) - df[col2].fillna(0)).clip(lower=0)


def add_ratio_column(df, numerator_col, denominator_col):
    """
    Возвращает Series: отношение numerator_col / denominator_col
    с защитой от деления на 0, заменой NaN на 0 и ограничением [0, 1].
    """
    num = df[numerator_col].fillna(0)
    denom = df[denominator_col].replace(0, pd.NA).fillna(pd.NA)
    ratio = num / denom
    return ratio.fillna(0).clip(0, 1)

In [256]:
df_osi_operations

,Equipment,Event,Starttime,Endtime
0,DBKAZ40 BL №1,RP_REPAIR,2036-08-19 11:00:00,2036-09-01 16:00:00
1,DBKAZ40 BL №1,CV_LONG_STOP,2035-04-27 07:00:00,2035-08-15 07:00:00
2,DBKAZ40 BL №1,WT_PPR,2035-02-17 07:00:00,2035-02-17 08:00:00
3,DBKAZ40 BL №1,WT_LUNCH,2035-02-17 11:00:00,2035-02-17 12:00:00
4,DBKAZ40 BL №1,WT_PPR,2035-02-17 19:00:00,2035-02-17 20:00:00
...,...,...,...,...
567646,ЯЯЯЯ-ФЕЙК,DR_CORE_DRILLING,2039-09-29 20:00:00,2039-09-29 22:30:00
567647,ЯЯЯЯ-ФЕЙК,DR_REAMING,2039-09-30 10:30:00,2039-09-30 12:30:00
567648,ЯЯЯЯ-ФЕЙК,DR_PUMP_REPLACEMENT,2039-09-30 13:00:00,2039-09-30 14:30:00
567649,ЯЯЯЯ-ФЕЙК,DR_WELL_FLUSHING,2039-10-01 11:30:00,2039-10-01 14:00:00


In [257]:
# Работа с df_operations
df_operations = df_operations[['Event', 'Category']] # pyright: ignore[reportOptionalSubscript]
df_operations = df_operations[df_operations['Category'].notna()].reset_index(drop=True)

In [258]:
if df_rigs is not None:
    df_rigs['Rig-Osidem'] = df_rigs['Rig-Osidem'].str.strip().str.lower()

In [259]:
if df_osi_operations is not None:
    df_osi_operations['Equipment'] = df_osi_operations['Equipment'].str.strip().str.lower()

In [260]:
rows_out = []

for row in df_osi_operations.itertuples(index=False):
    start = row.Starttime
    end = row.Endtime
    total_seconds = (end - start).total_seconds()

    if start.year == end.year and start.month == end.month:
        # Месяц не меняется — ничего не делим
        rows_out.append({
            **row._asdict(),
            'Year-Month': start.strftime('%Y-%m'),
            'Duration': total_seconds / 3600
        })
    else:
        # Делаем разбивку
        current_start = start
        while current_start < end:
            next_month = (current_start.replace(day=1) + timedelta(days=32)).replace(day=1)
            segment_end = min(end, next_month)
            segment_seconds = (segment_end - current_start).total_seconds()

            rows_out.append({
                **row._asdict(),
                'Starttime': current_start,
                'Endtime': segment_end,
                'Year-Month': current_start.strftime('%Y-%m'),
                'Duration': round((segment_seconds / total_seconds) * ((end - start).total_seconds() / 3600), 2)
            })

            current_start = segment_end
            
            


df_osi_operations = pd.DataFrame(rows_out)




In [261]:
df_osi_operations

,Equipment,Event,Starttime,Endtime,Year-Month,Duration
0,dbkaz40 bl №1,RP_REPAIR,2036-08-19 11:00:00,2036-09-01 11:00:00,2036-08,312.0
1,dbkaz40 bl №1,RP_REPAIR,2036-09-01 11:00:00,2036-09-01 16:00:00,2036-09,5.0
2,dbkaz40 bl №1,CV_LONG_STOP,2035-04-27 07:00:00,2035-05-01 07:00:00,2035-04,96.0
3,dbkaz40 bl №1,CV_LONG_STOP,2035-05-01 07:00:00,2035-06-01 07:00:00,2035-05,744.0
4,dbkaz40 bl №1,CV_LONG_STOP,2035-06-01 07:00:00,2035-07-01 07:00:00,2035-06,720.0
...,...,...,...,...,...,...
567917,яяяя-фейк,DR_CORE_DRILLING,2039-09-29 20:00:00,2039-09-29 22:30:00,2039-09,2.5
567918,яяяя-фейк,DR_REAMING,2039-09-30 10:30:00,2039-09-30 12:30:00,2039-09,2.0
567919,яяяя-фейк,DR_PUMP_REPLACEMENT,2039-09-30 13:00:00,2039-09-30 14:30:00,2039-09,1.5
567920,яяяя-фейк,DR_WELL_FLUSHING,2039-10-01 11:30:00,2039-10-01 14:00:00,2039-10,2.5


In [262]:
df_osi_operations[df_osi_operations['Event'] == 'WT_LUNCH']

,Equipment,Event,Starttime,Endtime,Year-Month,Duration
8,dbkaz40 bl №1,WT_LUNCH,2035-02-17 11:00:00,2035-02-17 12:00:00,2035-02,1.0
13,dbkaz40 bl №1,WT_LUNCH,2035-02-17 23:00:00,2035-02-18 00:00:00,2035-02,1.0
18,dbkaz40 bl №1,WT_LUNCH,2035-02-18 23:00:00,2035-02-19 00:00:00,2035-02,1.0
20,dbkaz40 bl №1,WT_LUNCH,2035-02-19 12:00:00,2035-02-19 13:00:00,2035-02,1.0
22,dbkaz40 bl №1,WT_LUNCH,2035-02-20 00:00:00,2035-02-20 01:00:00,2035-02,1.0
...,...,...,...,...,...,...
564374,пбу змо-1500пс №6,WT_LUNCH,2040-07-06 00:00:00,2040-07-06 01:00:00,2040-07,1.0
564376,пбу змо-1500пс №6,WT_LUNCH,2040-07-06 12:00:00,2040-07-06 13:00:00,2040-07,1.0
564378,пбу змо-1500пс №6,WT_LUNCH,2040-07-07 00:00:00,2040-07-07 01:00:00,2040-07,1.0
564810,пбу змо-1500пс №6,WT_LUNCH,2038-11-04 11:00:00,2038-11-04 13:00:00,2038-11,2.0


In [263]:
# Слияние
if df_osi_operations is not None and df_rigs is not None:
    df_merged = df_osi_operations.merge( # pyright: ignore[reportOptionalMemberAccess]
        df_rigs,
        left_on='Equipment',
        right_on='Rig-Osidem',
        how='left'
    )

In [264]:
df_merged = df_merged[df_merged['Rig-Acquire'].notna()].reset_index(drop=True)
df_temp = df_merged.drop(columns=['Equipment', 'Rig-Osidem'])

In [265]:
# Очистка колонок event
df_temp['Event'] = df_temp['Event'].str.strip().str.lower()
df_operations['Event'] = df_operations['Event'].str.strip().str.lower()

In [266]:
# Слияние с категориями
df_operations_total = df_temp.merge(df_operations, on='Event', how='left')
df_operations_total['Category'] = df_operations_total['Category'].fillna('Standard Work')

In [267]:
# Добавляем year и month
df_operations_total['Year-Month'] = df_operations_total['Starttime'].dt.strftime('%Y-%m')

In [268]:
df_operations_total

,Event,Starttime,Endtime,Year-Month,Duration,Drillcompany,Rig-Acquire,Tipe Of Circulation,Category
0,rp_repair,2036-08-19 11:00:00,2036-09-01 11:00:00,2036-08,312.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
1,rp_repair,2036-09-01 11:00:00,2036-09-01 16:00:00,2036-09,5.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
2,cv_long_stop,2035-04-27 07:00:00,2035-05-01 07:00:00,2035-04,96.0,KATCO,DBKAZ40-1,RC,Standard Work
3,cv_long_stop,2035-05-01 07:00:00,2035-06-01 07:00:00,2035-05,744.0,KATCO,DBKAZ40-1,RC,Standard Work
4,cv_long_stop,2035-06-01 07:00:00,2035-07-01 07:00:00,2035-06,720.0,KATCO,DBKAZ40-1,RC,Standard Work
...,...,...,...,...,...,...,...,...,...
528921,as_mast_down,2040-07-06 09:00:00,2040-07-06 12:00:00,2040-07,3.0,KATCO,ZMO1500-6-KAT,Direct,Standard Work
528922,as_mast_down,2040-07-06 13:00:00,2040-07-06 20:00:00,2040-07,7.0,KATCO,ZMO1500-6-KAT,Direct,Standard Work
528923,tr_rig_transport,2040-07-06 21:00:00,2040-07-07 00:00:00,2040-07,3.0,KATCO,ZMO1500-6-KAT,Direct,Standard Work
528924,tr_rig_transport,2040-07-07 01:00:00,2040-07-07 08:00:00,2040-07,7.0,KATCO,ZMO1500-6-KAT,Direct,Standard Work


In [269]:
df_operations_total = df_operations_total[df_operations_total['Event'] != 'wt_lunch']

# Получаем уникальные Year-Month и Rig-Acquire
unique_combinations = (
    df_operations_total[
        ['Year-Month', 'Rig-Acquire', 'Drillcompany', 'Tipe Of Circulation']
    ]
    .drop_duplicates()
)




# Список строк с ланчами
lunch_rows = []

for _, row in unique_combinations.iterrows():
    ym = row['Year-Month']
    rig = row['Rig-Acquire']
    company = row['Drillcompany']
    circulation = row['Tipe Of Circulation']
    
    year, month = map(int, ym.split('-'))
    days_in_month = calendar.monthrange(year, month)[1]
    
    duration = days_in_month * 2  # по 2 часа в день
    start_time = pd.Timestamp(f"{ym}-01 13:00:00")
    end_time = start_time + pd.Timedelta(hours=duration)

    lunch_rows.append({
        'Event': 'wt_lunch',
        'Starttime': start_time,
        'Endtime': end_time,
        'Duration': duration,
        'Year-Month': ym,
        'Drillcompany': company,
        'Rig-Acquire': rig,
        'Tipe Of Circulation': circulation,
        'Category': 'Planned_downtime',
    })

# Создаём DataFrame и добавляем к основному
df_lunch = pd.DataFrame(lunch_rows)
df_operations_total = pd.concat([df_operations_total, df_lunch], ignore_index=True)

# Сброс индекса
df_operations_total = df_operations_total.reset_index(drop=True)




In [270]:
df_operations_total

,Event,Starttime,Endtime,Year-Month,Duration,Drillcompany,Rig-Acquire,Tipe Of Circulation,Category
0,rp_repair,2036-08-19 11:00:00,2036-09-01 11:00:00,2036-08,312.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
1,rp_repair,2036-09-01 11:00:00,2036-09-01 16:00:00,2036-09,5.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
2,cv_long_stop,2035-04-27 07:00:00,2035-05-01 07:00:00,2035-04,96.0,KATCO,DBKAZ40-1,RC,Standard Work
3,cv_long_stop,2035-05-01 07:00:00,2035-06-01 07:00:00,2035-05,744.0,KATCO,DBKAZ40-1,RC,Standard Work
4,cv_long_stop,2035-06-01 07:00:00,2035-07-01 07:00:00,2035-06,720.0,KATCO,DBKAZ40-1,RC,Standard Work
...,...,...,...,...,...,...,...,...,...
515933,wt_lunch,2040-03-01 13:00:00,2040-03-04 03:00:00,2040-03,62.0,KATCO,ZMO1500-6-KAT,Direct,Planned_downtime
515934,wt_lunch,2040-04-01 13:00:00,2040-04-04 01:00:00,2040-04,60.0,KATCO,ZMO1500-6-KAT,Direct,Planned_downtime
515935,wt_lunch,2040-05-01 13:00:00,2040-05-04 03:00:00,2040-05,62.0,KATCO,ZMO1500-6-KAT,Direct,Planned_downtime
515936,wt_lunch,2040-06-01 13:00:00,2040-06-04 01:00:00,2040-06,60.0,KATCO,ZMO1500-6-KAT,Direct,Planned_downtime


In [271]:
# Сводная таблица pivot
pivot_df = df_operations_total.pivot_table(
    index=['Drillcompany', 'Rig-Acquire', 'Year-Month', 'Tipe Of Circulation'],
    columns='Category',
    values='Duration',
    aggfunc='sum',
    fill_value=0
).reset_index()


In [272]:
pivot_df

Category,Drillcompany,Rig-Acquire,Year-Month,Tipe Of Circulation,Planned_downtime,Standard Work,Unplanned_downtime_losses
0,BurGeoProekt,ZIF1200_01-BGP,2035-02,Direct,56.0,8.0,276.0
1,BurGeoProekt,ZIF1200_01-BGP,2035-03,Direct,109.0,423.0,278.0
2,BurGeoProekt,ZIF1200_01-BGP,2035-04,Direct,77.0,675.0,28.0
3,BurGeoProekt,ZIF1200_01-BGP,2035-05,Direct,62.0,744.0,0.0
4,BurGeoProekt,ZIF1200_01-BGP,2035-06,Direct,60.0,720.0,0.0
...,...,...,...,...,...,...,...
2986,TechnoService-Eng,ZIF1200_16-TSE,2038-03,Direct,151.0,523.0,90.0
2987,TechnoService-Eng,ZIF1200_16-TSE,2038-04,Direct,139.0,448.0,118.0
2988,TechnoService-Eng,ZIF1200_16-TSE,2038-05,Direct,134.0,499.0,109.0
2989,TechnoService-Eng,ZIF1200_16-TSE,2038-06,Direct,141.0,401.0,133.0


In [273]:
pivot_df = pivot_df.sort_values(
    by=['Drillcompany', 'Rig-Acquire', 'Year-Month'],
    ascending=[True, True, True]
).reset_index(drop=True)

# Функция для подсчёта часов в месяце
def hours_in_month(row):
    year_month_str = row['Year-Month']  # предполагается, что у вас есть колонка с таким именем
    year, month = map(int, year_month_str.split('-'))
    days = calendar.monthrange(year, month)[1]
    return days * 24


pivot_df['H In Month'] = pivot_df.apply(hours_in_month, axis=1)


# Преобразование ENDDATE в datetime
if df_meters is not None and df_rigs is not None:
    df_meters['Enddate'] = pd.to_datetime(df_meters['Enddate'], errors='coerce')
    df_meters.dropna(subset=['Enddate'], inplace=True)
    df_meters['Year-Month'] = df_meters['Enddate'].dt.strftime('%Y-%m')




C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\528234722.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meters['Enddate'] = pd.to_datetime(df_meters['Enddate'], errors='coerce')
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\528234722.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meters.dropna(subset=['Enddate'], inplace=True)
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\528234722.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [274]:
if df_meters is not None:
    pivot2 = pd.pivot_table(
        df_meters,
        index=['Year-Month', 'Drillrig'],
        columns='Holestatus',
        values='Holeid',
        aggfunc='count',
        fill_value=0
    )


if df_meters is not None:
    depth_sum = df_meters.groupby(['Year-Month', 'Drillrig'])['Depth'].sum()
    
    
pivot2['Depth'] = depth_sum
meters_pivot = pivot2.reset_index()

# Приведение к верхнему регистру и очистка пробелов
pivot_df['Rig-Acquire'] = pivot_df['Rig-Acquire'].str.strip().str.upper()
meters_pivot['Drillrig'] = meters_pivot['Drillrig'].str.strip().str.upper()

total_merged_df = pd.merge(
    pivot_df,
    meters_pivot,
    left_on=['Rig-Acquire', 'Year-Month'],
    right_on=['Drillrig', 'Year-Month'],
    how='left'
)


In [275]:
total_merged_df.drop(columns='Rig-Acquire', inplace=True)


# Расчёты Planned Production Time и Planned Factor с ограничениями
total_merged_df['Planned Production Time'] = add_difference_column(total_merged_df, 'H In Month', 'Planned_downtime')


total_merged_df['Planned Factor'] = add_ratio_column(
    total_merged_df, 'Planned Production Time', 'H In Month'
)

# Gross Operating Time (GOT) и Availability
total_merged_df['Gross Operating Time'] = add_difference_column(
    total_merged_df, 'Planned Production Time', 'Unplanned_downtime_losses'
)


In [276]:
total_merged_df['Availability'] = add_ratio_column(
    total_merged_df, 'Gross Operating Time', 'Planned Production Time'
)

# Назначаем коэффициенты по типу циркуляции
if df_circ_data is not None and 'Circ' in df_circ_data.columns and 'Standard Avarage Drilling, M/H' in df_circ_data.columns:
    circ_avg_drilling = df_circ_data.set_index('Circ')['Standard Avarage Drilling, M/H'].to_dict()
else:
    circ_avg_drilling = {}
    
    
if df_circ_data is not None and 'Circ' in df_circ_data.columns and 'Time To Well Drill, H' in df_circ_data.columns:
    time_to_well_drill = df_circ_data.set_index('Circ')['Time To Well Drill, H'].to_dict()
else:
    time_to_well_drill = {}


In [277]:
# Записываем два столбца в df
total_merged_df['Circulation Coeff'] = total_merged_df['Tipe Of Circulation'].map(circ_avg_drilling)
total_merged_df['Well Drill Coef'] = total_merged_df['Tipe Of Circulation'].map(time_to_well_drill)


# Потенциальная глубина бурения
total_merged_df['Potential Depth'] = (
    total_merged_df['Gross Operating Time'] * total_merged_df['Circulation Coeff']
)


total_merged_df['Net Operating Time'] = total_merged_df['Depth'].div(
    total_merged_df['Circulation Coeff']
).fillna(0)


total_merged_df['Speed Losses'] = add_difference_column(
    total_merged_df, 'Gross Operating Time', 'Net Operating Time'
)



In [278]:
total_merged_df['Performance'] = add_ratio_column(total_merged_df, 'Net Operating Time', 'Gross Operating Time')

total_merged_df['Quality Losses'] =  total_merged_df.get('Well Drill Coef', 0) * total_merged_df.get('LIQUID', 0)

# Valuable Operating Time
total_merged_df['Valuable Operating Time'] = add_difference_column(
    total_merged_df, 'Net Operating Time', 'Quality Losses'
    )



total_merged_df['Quality'] = add_ratio_column(
    total_merged_df, 'Valuable Operating Time', 'Net Operating Time'
    )


total_merged_df['OEE'] = (
    total_merged_df['Availability'] *
    total_merged_df['Performance'] *
    total_merged_df['Quality']
)


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\516393728.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\516393728.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return ratio.fillna(0).clip(0, 1)


In [279]:
total_merged_df['TRS'] = total_merged_df['OEE'] * total_merged_df['Planned Factor']


total_merged_df.columns = clean_column_names(total_merged_df.columns)


total_merged_df = total_merged_df.rename(columns={'Drillrig': 'Rig'})


df_losses = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Downtime',
        'Unplanned Downtime Losses',
        'Speed Losses',
        'Quality Losses'
    ],
    var_name='Loss Type',
    value_name='Loss Value'
)


In [280]:
df_productivity = pd.melt(
    total_merged_df,
    id_vars=[
        'Drillcompany',
        'Rig',
        'Year-Month',
        'Tipe Of Circulation'
    ],
    value_vars=[
        'Planned Factor',
        'Availability',
        'Performance',
        'Quality',
        'Oee',
        'Trs'
    ],
    var_name='Productivity Type',
    value_name='Productivity Value'
)


df_operations_total['Category'] = df_operations_total['Category'].replace({
    'Planned_downtime': 'Planned Downtime',
    'Unplanned_downtime_losses': 'Unplanned Downtime Losses'
})


df_operations_total = df_operations_total.rename(columns={'Rig-Acquire': 'Rig'})


df_operations_total['Event Category'] = df_operations_total['Event'].str.split('_').str[0].str.upper()
df_operations_total['Event'] = df_operations_total['Event'].str.split('_', n=1).str[1].str.capitalize()


df_events_duration = df_operations_total[
    [
        'Year-Month',
        "Category",
        'Drillcompany',
        'Rig',
        'Tipe Of Circulation',
        "Event Category",
        "Event",
        "Duration"
    ]
].sort_values(
    by=['Year-Month', 'Rig', 'Category', 'Duration'],
    ascending=[True, True, True, False],
)


In [281]:
if df_meters is not None:
    df_meters.dropna(subset=['Drillcompany', 'Drillrig'], inplace=True)
    
    df_meters.columns = [
    'Hole ID',
    'Drilling Company',
    'Rig',
    'Purpose',
    'Status',
    'End_Date',
    'Depth_m',
    'Year-Month'
]

    df_meters = df_meters.drop(columns=['End_Date'])


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_26672\3178993040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meters.dropna(subset=['Drillcompany', 'Drillrig'], inplace=True)


In [282]:
unique_dates_series = pd.Series(
    sorted(df_productivity['Year-Month'].dropna().unique()),
    name='Year-Month'
)

unique_companies_series = pd.Series(
    sorted(df_productivity['Drillcompany'].dropna().unique()),
    name='Companies'
)

unique_rigs_series = pd.Series(
    sorted(df_productivity['Rig'].dropna().unique()),
    name='Rig'
)

unique_circ_series = pd.Series(
    sorted(df_productivity['Tipe Of Circulation'].dropna().unique()),
    name='Circ Type'
)

unique_product_type_series = pd.Series(
    sorted(df_productivity['Productivity Type'].dropna().unique()),
    name='Productivity Type'
)

unique_losses_series = pd.Series(
    sorted(df_losses['Loss Type'].dropna().unique()),
    name='Loss Type'
)


In [ ]:
folder = r'k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/'

save_df_to_csv(df_meters, 'df_meters.csv', folder)
save_df_to_csv(df_events_duration, 'df_events_duration.csv', folder)
save_df_to_csv(df_productivity, 'df_productivity.csv', folder)
save_df_to_csv(df_losses, 'df_losses.csv', folder)
save_df_to_csv(unique_dates_series, 'df_dates.csv', folder)

Файл сохранён: k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/df_meters.csv
Файл сохранён: k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/df_events_duration.csv
Файл сохранён: k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/df_productivity.csv
Файл сохранён: k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/df_losses.csv
Файл сохранён: k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/extr_csv_files/df_dates.csv


: 